# Model Selection & Training

**Objective:** Train and tune models to predict Airbnb new user booking destinations.

In this notebook, we will:
1. Load the preprocessed data
2. Split the data into training and validation sets
3. Train various machine learning models
4. Tune hyperparameters using cross-validation
5. Save the trained models for later evaluation

## 1. Import Required Libraries

In [ ]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from datetime import datetime

# Import ML libraries
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

# For hyperparameter optimization
import optuna

# Visualization settings
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid", palette="muted")

# Display progress bars for longer operations
from tqdm.notebook import tqdm

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## 2. Load Preprocessed Data

In [ ]:
# Load the preprocessed training data
processed_data_path = '../data/processed/processed_train_users.csv'

if os.path.exists(processed_data_path):
    df = pd.read_csv(processed_data_path)
    print(f"Loaded preprocessed data with shape: {df.shape}")
else:
    print(f"Error: Preprocessed data file not found at {processed_data_path}")
    print("Please run the 02_preprocessing.ipynb notebook first.")

In [ ]:
# Quick check of the data
df.head()

In [ ]:
# Distribution of target variable
plt.figure(figsize=(12, 6))
target_counts = df['country_destination'].value_counts()
sns.barplot(x=target_counts.index, y=target_counts.values)
plt.title('Distribution of Country Destinations')
plt.xlabel('Country Destination')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

# Print percentages
print("Class distribution:")
print(df['country_destination'].value_counts(normalize=True) * 100)

## 3. Prepare Data for Training

In [ ]:
# Separate features and target variable
X = df.drop(['country_destination', 'id'], axis=1, errors='ignore')  # Drop target and user id
y = df['country_destination']

# Check if there are any remaining non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=['number']).columns.tolist()
print(f"Non-numeric columns that need encoding: {non_numeric_cols}")

# If there are categorical columns that weren't one-hot encoded in preprocessing
if non_numeric_cols:
    X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)
    print(f"After one-hot encoding, X shape: {X.shape}")

In [ ]:
# Split data into training and validation sets using stratified sampling
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}")

In [ ]:
# Check for any NaN values
print(f"NaN values in training features: {X_train.isna().sum().sum()}")
print(f"NaN values in validation features: {X_val.isna().sum().sum()}")

# Replace any remaining NaN values with 0
if X_train.isna().sum().sum() > 0:
    X_train = X_train.fillna(0)
    X_val = X_val.fillna(0)

In [ ]:
# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Save the scaler for future use
os.makedirs('../models', exist_ok=True)
with open('../models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## 4. Baseline Models

We'll start with simple baseline models before moving to more complex ones.

In [ ]:
# Function to evaluate and display model performance
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='weighted')
    
    # Print results
    print(f"\n{model_name} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score (weighted): {f1:.4f}")
    
    # Display confusion matrix
    cm = confusion_matrix(y_val, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    # Display classification report
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred))
    
    return model, accuracy, f1

In [ ]:
# 4.1 Logistic Regression (Baseline)
lr_model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', random_state=42)
lr_model, lr_accuracy, lr_f1 = evaluate_model(lr_model, X_train_scaled, X_val_scaled, y_train, y_val, "Logistic Regression")

In [ ]:
# 4.2 Random Forest (Baseline)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model, rf_accuracy, rf_f1 = evaluate_model(rf_model, X_train, X_val, y_train, y_val, "Random Forest")

In [ ]:
# 4.3 XGBoost (Baseline)
xgb_model = xgb.XGBClassifier(objective='multi:softmax', random_state=42, n_jobs=-1)
xgb_model, xgb_accuracy, xgb_f1 = evaluate_model(xgb_model, X_train, X_val, y_train, y_val, "XGBoost")

In [ ]:
# 4.4 LightGBM (Baseline)
lgb_model = lgb.LGBMClassifier(objective='multiclass', random_state=42, n_jobs=-1)
lgb_model, lgb_accuracy, lgb_f1 = evaluate_model(lgb_model, X_train, X_val, y_train, y_val, "LightGBM")

In [ ]:
# Compare baseline models
baseline_results = {
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', 'LightGBM'],
    'Accuracy': [lr_accuracy, rf_accuracy, xgb_accuracy, lgb_accuracy],
    'F1 Score': [lr_f1, rf_f1, xgb_f1, lgb_f1]
}

baseline_df = pd.DataFrame(baseline_results)
baseline_df = baseline_df.sort_values('F1 Score', ascending=False).reset_index(drop=True)
baseline_df

In [ ]:
# Visualize baseline model comparison
plt.figure(figsize=(12, 6))

x = baseline_df['Model']
width = 0.35

plt.bar(x=np.arange(len(x)), height=baseline_df['Accuracy'], width=width, label='Accuracy', alpha=0.7)
plt.bar(x=np.arange(len(x)) + width, height=baseline_df['F1 Score'], width=width, label='F1 Score', alpha=0.7)

plt.xlabel('Model')
plt.ylabel('Score')
plt.title('Baseline Model Performance Comparison')
plt.xticks(np.arange(len(x)) + width/2, x, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

## 5. Feature Importance Analysis

Let's analyze feature importance for the best-performing model from our baselines.

In [ ]:
# Function to plot feature importance
def plot_feature_importance(model, feature_names, model_name, top_n=20):
    if hasattr(model, 'feature_importances_'):
        # For tree-based models
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        # For linear models like Logistic Regression
        importances = np.mean(np.abs(model.coef_), axis=0)
    else:
        print(f"Model {model_name} does not have feature importances available.")
        return
    
    # Get feature importances and names
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })
    
    # Sort by importance and get top N
    feature_importance = feature_importance.sort_values('Importance', ascending=False).head(top_n)
    
    # Plot
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance)
    plt.title(f'Top {top_n} Feature Importance - {model_name}')
    plt.tight_layout()
    plt.show()
    
    return feature_importance

In [ ]:
# Identify the best model from baselines
best_baseline_model_name = baseline_df.iloc[0]['Model']
print(f"Best performing baseline model: {best_baseline_model_name}")

# Get the corresponding model object
if best_baseline_model_name == 'Logistic Regression':
    best_model = lr_model
elif best_baseline_model_name == 'Random Forest':
    best_model = rf_model
elif best_baseline_model_name == 'XGBoost':
    best_model = xgb_model
else:  # LightGBM
    best_model = lgb_model

# Plot feature importance for the best model
feature_importance_df = plot_feature_importance(best_model, X_train.columns, best_baseline_model_name)

## 6. Hyperparameter Tuning with Cross-Validation

Let's tune the hyperparameters of our best-performing model to improve its performance.

In [ ]:
# Find best baseline model to tune
best_model_name = best_baseline_model_name
print(f"Tuning hyperparameters for: {best_model_name}")

In [ ]:
# 6.1 Tuning Random Forest (if it's the best model)
if best_model_name == 'Random Forest':
    # Define parameter grid
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Create model
    rf = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    # Use cross-validation with grid search
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, 
                              scoring='f1_weighted', n_jobs=-1, verbose=1)
    
    # Train with grid search
    grid_search.fit(X_train, y_train)
    
    # Get best model and parameters
    best_rf = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # Evaluate tuned model
    tuned_model, tuned_accuracy, tuned_f1 = evaluate_model(best_rf, X_train, X_val, y_train, y_val, "Tuned Random Forest")

In [ ]:
# 6.2 Tuning XGBoost (if it's the best model)
if best_model_name == 'XGBoost':
    # Define parameter grid
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'n_estimators': [100, 200, 300],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9]
    }
    
    # Create model
    xgb_clf = xgb.XGBClassifier(objective='multi:softmax', random_state=42, n_jobs=-1)
    
    # Use cross-validation with grid search
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=cv, 
                              scoring='f1_weighted', n_jobs=-1, verbose=1)
    
    # Train with grid search
    grid_search.fit(X_train, y_train)
    
    # Get best model and parameters
    best_xgb = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # Evaluate tuned model
    tuned_model, tuned_accuracy, tuned_f1 = evaluate_model(best_xgb, X_train, X_val, y_train, y_val, "Tuned XGBoost")

In [ ]:
# 6.3 Tuning LightGBM (if it's the best model)
if best_model_name == 'LightGBM':
    # Define parameter grid
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, -1],  # -1 means no limit
        'num_leaves': [31, 63, 127],
        'subsample': [0.7, 0.8, 0.9]
    }
    
    # Create model
    lgbm = lgb.LGBMClassifier(objective='multiclass', random_state=42, n_jobs=-1)
    
    # Use cross-validation with grid search
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=cv, 
                              scoring='f1_weighted', n_jobs=-1, verbose=1)
    
    # Train with grid search
    grid_search.fit(X_train, y_train)
    
    # Get best model and parameters
    best_lgbm = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # Evaluate tuned model
    tuned_model, tuned_accuracy, tuned_f1 = evaluate_model(best_lgbm, X_train, X_val, y_train, y_val, "Tuned LightGBM")

In [ ]:
# 6.4 Tuning Logistic Regression (if it's the best model)
if best_model_name == 'Logistic Regression':
    # Define parameter grid
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'newton-cg', 'sag'],
        'penalty': ['l2', 'none']
    }
    
    # Create model
    lr = LogisticRegression(max_iter=2000, multi_class='multinomial', random_state=42)
    
    # Use cross-validation with grid search
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=cv, 
                              scoring='f1_weighted', n_jobs=-1, verbose=1)
    
    # Train with grid search
    grid_search.fit(X_train_scaled, y_train)
    
    # Get best model and parameters
    best_lr = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # Evaluate tuned model
    tuned_model, tuned_accuracy, tuned_f1 = evaluate_model(best_lr, X_train_scaled, X_val_scaled, y_train, y_val, "Tuned Logistic Regression")

## 7. Advanced Hyperparameter Tuning with Optuna

Now let's use Optuna to perform more sophisticated hyperparameter optimization for our best model.

In [ ]:
# Define the objective function for XGBoost
def objective_xgb(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
        'objective': 'multi:softmax',
        'random_state': 42
    }
    
    # Create model
    model = xgb.XGBClassifier(**param)
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1_weighted')
    
    # Return mean CV score
    return cv_scores.mean()

In [ ]:
# Define the objective function for LightGBM
def objective_lgb(trial):
    param = {
        'objective': 'multiclass',
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'random_state': 42
    }
    
    # Create model
    model = lgb.LGBMClassifier(**param)
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1_weighted')
    
    # Return mean CV score
    return cv_scores.mean()

In [ ]:
# Define the objective function for RandomForest
def objective_rf(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'random_state': 42
    }
    
    # Create model
    model = RandomForestClassifier(**param)
    
    # Perform cross-validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1_weighted')
    
    # Return mean CV score
    return cv_scores.mean()

In [ ]:
# Choose which objective function to use based on best baseline model
if best_model_name == 'XGBoost':
    objective_function = objective_xgb
    print("Running Optuna optimization for XGBoost...")
elif best_model_name == 'LightGBM':
    objective_function = objective_lgb
    print("Running Optuna optimization for LightGBM...")
elif best_model_name == 'Random Forest':
    objective_function = objective_rf
    print("Running Optuna optimization for Random Forest...")
else:
    # For Logistic Regression, we'll use GridSearchCV results
    print("Skipping Optuna optimization for Logistic Regression (GridSearchCV is sufficient)")

In [ ]:
# Run Optuna optimization if applicable
if best_model_name in ['XGBoost', 'LightGBM', 'Random Forest']:
    # Create study
    study = optuna.create_study(direction='maximize')
    
    # Optimize with 50 trials (adjust as needed)
    study.optimize(objective_function, n_trials=50)
    
    # Print best parameters
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value:.4f}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    
    # Create model with best parameters
    if best_model_name == 'XGBoost':
        final_model = xgb.XGBClassifier(
            objective='multi:softmax',
            random_state=42,
            **trial.params
        )
    elif best_model_name == 'LightGBM':
        final_model = lgb.LGBMClassifier(
            objective='multiclass',
            random_state=42,
            **trial.params
        )
    else:  # Random Forest
        final_model = RandomForestClassifier(
            random_state=42,
            **trial.params
        )
    
    # Evaluate final model
    final_model, final_accuracy, final_f1 = evaluate_model(
        final_model, X_train, X_val, y_train, y_val, f"Optuna-Tuned {best_model_name}"
    )
else:
    # For Logistic Regression, use the GridSearchCV results as the final model
    final_model = best_lr if 'best_lr' in locals() else lr_model
    final_accuracy = tuned_accuracy if 'tuned_accuracy' in locals() else lr_accuracy
    final_f1 = tuned_f1 if 'tuned_f1' in locals() else lr_f1

## 8. Retrain Final Model on Full Training Data

In [ ]:
# Retrain final model on the full training dataset
print(f"Training final {best_model_name} model on full training data...")
final_model.fit(X_train, y_train)

In [ ]:
# Save the final model
model_filename = f"../models/final_{best_model_name.lower().replace(' ', '_')}_model.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(final_model, f)

print(f"Final model saved to {model_filename}")

In [ ]:
# Save model metadata
model_meta = {
    'model_type': best_model_name,
    'accuracy': final_accuracy,
    'f1_score': final_f1,
    'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'model_file': model_filename,
    'feature_count': X_train.shape[1],
    'train_samples': X_train.shape[0],
    'validation_samples': X_val.shape[0],
    'classes': list(final_model.classes_)
}

meta_filename = "../models/model_metadata.json"
import json
with open(meta_filename, 'w') as f:
    json.dump(model_meta, f, indent=4)

print(f"Model metadata saved to {meta_filename}")

## 9. Final Performance on Validation Set

In [ ]:
# Make predictions on validation set
y_pred = final_model.predict(X_val)

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='weighted')

print(f"Final Model ({best_model_name}) Performance on Validation Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (weighted): {f1:.4f}")

In [ ]:
# Display final confusion matrix
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=final_model.classes_, yticklabels=final_model.classes_)
plt.title(f'Final Confusion Matrix - {best_model_name}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(f"../outputs/final_confusion_matrix.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Display classification report
print("Final Classification Report:")
print(classification_report(y_val, y_pred))

## 10. Predict Top 5 Countries (Kaggle Evaluation Metric)

The Kaggle competition uses NDCG@5 (Normalized Discounted Cumulative Gain) as the evaluation metric, so we need to predict the top 5 most likely destinations for each user.

In [ ]:
# Function to get top 5 predictions for each sample
def get_top5_predictions(model, X):
    # Get probability scores for each class
    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(X)
    else:
        # For models without predict_proba
        print("Warning: Model doesn't have predict_proba. Using alternative approach.")
        # Example alternative for XGBoost:
        if isinstance(model, xgb.XGBClassifier):
            proba = model.predict_proba(X)
        else:
            raise ValueError("Model doesn't support probability predictions")
    
    # Get classes
    classes = model.classes_
    
    # Get top 5 indices
    top5_indices = np.argsort(-proba, axis=1)[:, :5]
    
    # Convert indices to class labels
    top5_predictions = np.take(classes, top5_indices)
    
    return top5_predictions

In [ ]:
# Get top 5 predictions for validation set
top5_preds = get_top5_predictions(final_model, X_val)

# Check a few examples
for i in range(min(5, len(y_val))):
    print(f"Sample {i+1}:")
    print(f"True destination: {y_val.iloc[i]}")
    print(f"Top 5 predictions: {top5_preds[i]}")
    print("-" * 40)

In [ ]:
# Calculate how often the true label is in the top 5 predictions
hits = 0
for i, true_label in enumerate(y_val):
    if true_label in top5_preds[i]:
        hits += 1

top5_accuracy = hits / len(y_val)
print(f"Top-5 Accuracy: {top5_accuracy:.4f}")

## 11. Summary of Results

Let's summarize our model training process and results.

In [ ]:
# Create a summary DataFrame
summary = pd.DataFrame({
    'Metric': ['Accuracy', 'F1 Score (weighted)', 'Top-5 Accuracy'],
    'Value': [accuracy, f1, top5_accuracy]
})

print(f"Final Model: {best_model_name}")
summary

In [ ]:
# Plot final feature importance
if hasattr(final_model, 'feature_importances_') or hasattr(final_model, 'coef_'):
    feature_importance_df = plot_feature_importance(final_model, X_train.columns, f"Final {best_model_name}", top_n=20)
    
    # Save feature importance plot
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
    plt.title(f'Top 20 Feature Importance - Final {best_model_name}')
    plt.tight_layout()
    plt.savefig("../outputs/feature_importance.png", dpi=300, bbox_inches='tight')
    plt.close()

## 12. Save Model Pipeline for Inference

Finally, let's save the complete pipeline for making predictions on new users.

In [ ]:
# Create a dictionary with all components needed for inference
pipeline_components = {
    'model': final_model,
    'scaler': scaler,
    'feature_names': X_train.columns.tolist(),
    'target_classes': final_model.classes_.tolist()
}

# Save the pipeline
pipeline_filename = "../models/prediction_pipeline.pkl"
with open(pipeline_filename, 'wb') as f:
    pickle.dump(pipeline_components, f)

print(f"Prediction pipeline saved to {pipeline_filename}")

## 13. Next Steps

Here are the next steps for the Airbnb New User Booking Prediction project:

1. **Evaluation on Test Set**: Apply the trained model to the test set in `05_evaluation.ipynb`.
2. **Prepare Kaggle Submission**: Format predictions according to the competition requirements.
3. **Feature Engineering Extensions**: Consider additional features or engineering approaches:
   - Try more sophisticated session data aggregation
   - Extract more date-based features
   - Consider NLP features from text fields if available
4. **Ensemble Methods**: Combine multiple models for improved performance.
5. **Deep Learning**: Consider neural network approaches for this multiclass classification problem.

The code developed in this notebook provides a solid foundation for multiclass classification with comprehensive validation and evaluation metrics.